## Face recognition



Load the original images present in the files *'YaleB\_32x32.mat'*. This is a small part of the freely available Extended Yale Face Database B downloaded from http://www.cad.zju.edu.cn/home/dengcai/Data/FaceData.html. It contains 2414 cropped images resized to 32x32 pixels. Every image is represented as a vector 1x1024 and all images are stacked in a matrix called data. There are 38 subjects with around 64 near frontal images per individual under different illumination conditions. Once loaded and normalised the data, such that the pixels are between 0 and 1, you can plot images using the function *'imshow'*.

# Goal
The goal of this part is to evaluate the performance of the dimensionality reduction techniques presented this morning for face recognition. We divide the data-set into two parts, training and test. For every dimensionality reduction technique, you will first extract a set of basis images from your training data-set. Then, you will project the test subjects in this new basis and use the nearest neighbor algorithm to evaluate the performance of the dimensionality reduction technique. 

Let's load the data.

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google_drive_downloader import GoogleDriveDownloader as gdd
  gdd.download_file_from_google_drive(file_id='1rgICXtcIAgDqSoHnNXNZMD_iNABF3RZA',
  dest_path='./YaleB_32x32.mat')
else:
  print('You are not using Colab. Please define working_dir with the absolute path to the folder where you downloaded the data')

# Please modify working_dir only if you are using your Anaconda (and not Google Colab)
Working_directory="./"   

Load the libraries

In [ ]:
import math
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
plt.close('all')

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF


from scipy import linalg as LA
from scipy.stats import ortho_group

from scipy.io import loadmat

Here you can copy the functions `pcaLecture`, `KpcaGaussianLecture` and `FastICALecture` implemented in the previous jupyter-notebook.

In [ ]:
# For IMP
def pcaLecture(X):
    XXXXXXX

In [ ]:
# For IMP
def KpcaGaussianLecture(X,gamma):
    XXXXXXXX

In [ ]:
# For IMP
def FastICALecture(X,N_Iter=3000,tol=1e-5,plot_evolution=0,whiten=True):
    XXXXXXX

This is a useful function to plot the basis images. Be careful, each row of data is a basis image.

In [ ]:
def plotFaces(data,r,c,ncol=2,N=0,indeces=None,title=None):
    # data: each face is a row in data
    # r,c = number of rows and columns of each image
    # n_col = number of columns for subplots
    # N = random images to plot (used only if indeces is empty)
    # indeces = indeces of images to plot
    # title = title of the plot

   
    if indeces is None:
        if N==0:
            raise NameError('You should define either N or indeces')
        else:
            print('Use N random subjects')
            indeces=np.random.randint(0,data.shape[0],(N,1))
            
    nrow=math.ceil(len(indeces)/ncol)
    
    fig=plt.figure(figsize=(17, 6))
    plt.suptitle(title, size=16)
    for i, index in enumerate(indeces):
        fig.add_subplot(nrow, ncol, i+1)
        plt.imshow(np.resize(data[index,:],(r,c)).T,origin='upper',cmap='gray')
        plt.xticks(())
        plt.yticks(())
    #plt.subplots_adjust(left=0.01, bottom=0.05, right=0.99, top=0.93, wspace=0.04, hspace=0.0)
  

Let's load the data and compute some parameters.

In [ ]:
x = loadmat(Working_directory + './YaleB_32x32.mat')
data=x['fea']
d=data.shape[1] # number of pixels of the images
subjectIndex=x['gnd'] # we have one index per subject
maxValue = np.max(np.max(data)) # max intensity value
data = data/maxValue; # Scale pixels to [0,1]

Ns=len(np.unique(subjectIndex)); # Number subjects
Is=round(len(subjectIndex)/Ns) # Number images per subject (on average, not the same number for every subject)
r=int(np.sqrt(d)) # number rows of each image
c=r # number columns of each image, equal to row since images are square

print('There are', data.shape[0], 'facial images and each image has', d, 'pixels' )
print('There are', Ns, 'different subjects and each subject has on average', Is, 'images')

Let's plot first 10 images of different subjects and then 10 images of the same subject but with different positions and illumination conditions

In [ ]:
# Plot data      
indexDifferent=np.arange(1,Is*40,Is)     
plotFaces(data,r,c,ncol=3,indeces=indexDifferent[0:10],title='Different subjects')       
indexSame=np.arange(0,10,1)      
plotFaces(data,r,c,ncol=2,indeces=indexSame,title='Different positions of the same subjects')       


We can now use PCA to investigate the main variations within the data.

**Question:**
1. (IMP + IMH) How many modes do you need to explain at least 80% of the variability in the data ? Look at the three main modes and explain which are the main variations in the data.

In [ ]:
# Linear interpolation along the first two modes
Xm=data.mean(axis=0) # average face

pca = PCA(random_state=1) # by fixing the random_state we are sure that results are always the same
YpcaTrain=pca.fit_transform(data)
UpcaTrain=pca.components_.T # we want PC on columns
var_explained_pca=pca.explained_variance_ratio_
DpcaTrain = (pca.singular_values_)**2/(data.shape[0]-1) # computation of the eigenvalues
indices=np.linspace(-3, 3, num=7, dtype=np.int16) # Interpolation indices

# Variance explained by each eigenvector
fig=plt.figure(figsize=(7, 7))
ax=plt.subplot(111)
ax.set_xlim(0, 9)
dim=np.arange(0,10,1)
plt.plot(np.concatenate(([0], np.cumsum(var_explained_pca)*100)))
plt.xticks(dim)
plt.xlabel('Number of eigenvectors',fontsize=15)
plt.ylabel('% variance explained',fontsize=15)
plt.title('Variance explained by PCA modes',fontsize=17)

## First mode
fig=plt.figure(figsize=(17, 3))
plt.suptitle('Variations along the first mode of PCA', size=20)
for i, index in enumerate(indices):
  image = Xm + index * np.sqrt(DpcaTrain[0]) * UpcaTrain[:,0]
  fig.add_subplot(1, len(indices), i+1)
  plt.imshow(np.resize(image,(r,c)).T,origin='upper',cmap='gray')
  if index != 0:
    plt.xlabel(r'%i $\sigma$' %index, fontsize=15)
  else:
    plt.xlabel('Average face', fontsize=15)
  plt.xticks(())
  plt.yticks(())
  plt.subplots_adjust(left=0.01, bottom=0.05, right=0.99, top=0.93, wspace=0.04, hspace=0.0)


## Second mode
fig=plt.figure(figsize=(17, 3))
plt.suptitle('Variations along the second mode of PCA', size=20)
for i, index in enumerate(indices):
  image = Xm + index * np.sqrt(DpcaTrain[1]) * UpcaTrain[:,1]
  fig.add_subplot(1, len(indices), i+1)
  plt.imshow(np.resize(image,(r,c)).T,origin='upper',cmap='gray')
  if index != 0:
    plt.xlabel(r'%i $\sigma$' %index, fontsize=15)
  else:
    plt.xlabel('Average face', fontsize=15)
  plt.xticks(())
  plt.yticks(())
  plt.subplots_adjust(left=0.01, bottom=0.05, right=0.99, top=0.93, wspace=0.04, hspace=0.0)

## Third mode
fig=plt.figure(figsize=(17, 3))
plt.suptitle('Variations along the third mode of PCA', size=20)
for i, index in enumerate(indices):
  image = Xm + index * np.sqrt(DpcaTrain[2]) * UpcaTrain[:,2]
  fig.add_subplot(1, len(indices), i+1)
  plt.imshow(np.resize(image,(r,c)).T,origin='upper',cmap='gray')
  if index != 0:
    plt.xlabel(r'%i $\sigma$' %index, fontsize=15)
  else:
    plt.xlabel('Average face', fontsize=15)
  plt.xticks(())
  plt.yticks(())
  plt.subplots_adjust(left=0.01, bottom=0.05, right=0.99, top=0.93, wspace=0.04, hspace=0.0)

We can now move to evaluate the  performance of the dimensionality reduction techniques presented this morning for face recognition. We first divide the data-set into two parts, training (80%) and test (20%) in a stratified way (subjects are divided in a balanced way between the two parts).

We will see in the next lecture why we need to divide into training and test sets. For now, just know that we will use the first set to train our algorithm and the second set to test the performance of our algorithm on new, unseen data.

For every dimensionality reduction technique, you will first extract a set of basis images from your training data-set. Then, you will project the test subjects in this new basis and use the nearest neighbor algorithm
to evaluate the performance of the dimensionality reduction technique. For each test sample, the nearest neighbor algorithm simply looks for the closest training sample and then assigns the same label (i.e. index of subject).



In [ ]:
Xtrain, Xtest, Id_Train, Id_Test = train_test_split(data,subjectIndex,test_size=0.20,stratify=subjectIndex, random_state=44)
Xctest=Xtest-np.mean(Xtest,axis=0) # centering
Xctrain=Xtrain-np.mean(Xtrain,axis=0) # centering

As first idea, we could simply use the pixel intensities as features. This is basically like using the original data, without dimensionality reducton techniques.

In [ ]:
## Use the pixel intensities to find the correct subject for the test images
NN=KNeighborsClassifier(n_neighbors=1)
NN.fit(Xctrain,Id_Train.ravel())
print('By using the pixel intensities, we use ', Xctrain.shape[1], ' features')
print('Percentage of correct answer using the pixel intensities is ', NN.score(Xctest,Id_Test))


# PCA

You will first use PCA. Compute the scores $Y_{train}$, eigenvectors $U_{train}$ and eigenvalues $D_{train}$ of the training set. The eigenvectors $U_{train}$ represent the basis images and they are usually called *'Eigenfaces'*. Then, project both training and test data onto the eigenvectors that explain 99$\%$ of the variability of the training set $L_{train}^{99}$. You will obtain two vectors of scores, $Y_{train}^{99}=X_{train} L_{train}^{99}$ and $Y_{test}^{99}=X_{test} L_{test}^{99}$, which you will use for evaluating the performance of the algorithm. Use the function `KNeighborsClassifier` to test the performance.

**Practical Questions:** 

1. (IMP+IMH) Use either the scikit-learn implementation or yours (better!) to compute the PCA for the training data-set. Comment the eigenfaces. Do they seem "real" ?

**Theoretical Questions:** 

1. (IMP+IMH) In your opinion, why do we need to center the data before computing a PCA ? If you want, you can use the previous toy examples to answer this question.
2. (IMP+IMH) Let $X$ be the original data, a matrix $\left[ N,d \right]$, and $Y$ the scores of a PCA keeping all eigenvectors, which means that $Y$ is also a matrix $\left[ N,d \right]$. Are $X$ and $Y$ equal ? If not, why ? What would you use (generally speaking) in a machine learning problem ? Why ?
3. (IMP) Let $x_p$ and $x_q$ be two row-vectors representing two images, $U$ an *orthogonal* matrix whose columns are the eigenvectors of $X$ and $y_p=x_pU$, $y_q=x_qU$, check that $x_px_q^T=y_py_q^T$. This shows that $Y=XU$ is a linear transformation that preserves inner products.
4. (IMP) Let $C$ be the covariance matrix of $X$ and $C=UDU^T$ its eigen decomposition. Show that the covariance matrix of $Y=XU$ is $D$.

In [ ]:
## PCA (scikit-learn implementation)
pca = PCA(random_state=1) # by fixing the random_state we are sure that results are always the same
YpcaTrain=pca.fit_transform(Xtrain)
UpcaTrain=pca.components_.T # we want PC on columns
var_explained_pca=pca.explained_variance_ratio_

# Threshold defined as 99% of the variability
Threshold_PCA = 0.99
CumulativePca=np.cumsum(var_explained_pca)
indexPCA=np.argwhere(CumulativePca>Threshold_PCA)
PCAComp=indexPCA[0][0]

print('PCA uses ', PCAComp, ' features')

# Selection of the eigenvectors 
Yr_train_PCA=YpcaTrain[:,:PCAComp]
Ur_train_PCA=UpcaTrain[:,:PCAComp]

# Computation of the test scores using the eigenvectors computed with the
# training data-set
Yr_test_PCA=np.dot(Xctest,Ur_train_PCA)

# Plot the Eigenfaces
plotFaces(UpcaTrain.T,r,c,ncol=2,indeces=np.arange(0,10,1),title='PCA - Eigenfaces')       

# Score
NN.fit(Yr_train_PCA,Id_Train.ravel())
print('Percentage of correct answer using PCA is ', NN.score(Yr_test_PCA,Id_Test))


In [ ]:
# For IMP
## PCA (your implementation)
XXXXXXXXXXX

# KPCA

In this section, we are going to do exactly the same procedure as before but using Kernel-PCA with a Gaussian kernel. Remember that we need to compute and center the test kernel matrix $[\mathbf{\tilde{K}}]_{ij}=<\phi(x_i)-\frac{1}{N} \sum_{s=1}^N \phi(x_s),\phi(x_j)-\frac{1}{N} \sum_{s=1}^N \phi(x_s)>$ and that, once computed the basis vectors in the training set $\{  \mathbf{\alpha}_i \}$, we can compute the score for a test sample $t$ using the following equation:

$y_i(t) = \sum_{j=1}^N a_{ij} <\phi(t)-\frac{1}{N} \sum_{s=1}^N \phi(x_s),\phi(x_j)-\frac{1}{N} \sum_{s=1}^N \phi(x_s)> = \sum_{j=1}^N a_{ij} \tilde{k}(t,x_j)$

Answer the following questions:

**Questions:** 

1. (IMP+IMH) Look for the best gamma value
1. (IMP+IMH) Why the basis vectors $\{  \mathbf{\alpha}_i \}$ are not plotted as in PCA ?
2. (IMP+IMH) Is it worth it, in your opinion, to compute PCA and/or KPCA ? Why not using the original pixel intensities ? Please consider the following aspects in your answer: performance, computational time, number of features, and interpretability of the results.
3. (IMP - Optional) Create a new function `Kpca_poly_lecture` where you change the kernel to $k(x,y)=<x,y>^d$. Evaluate the performance of this new kernel.


In [ ]:
# Kernel-PCA (scikit-learn implementation)
## choose a gamma value
gamma=4
#

Kpca = KernelPCA(kernel='rbf', gamma=gamma, remove_zero_eig=True, random_state=1)
YKpca=Kpca.fit_transform(Xctrain)
DKpca=Kpca.lambdas_
AnKpca=Kpca.alphas_

# variance explained
tot=np.sum(np.real(DKpca))
varexplKpca = DKpca/tot # computation of explained variance

# Threshold defined as 99% of the variability
Threshold_KPCA = 0.99
CumulativeKPca=np.cumsum(varexplKpca)
indexKPCA=np.argwhere(CumulativeKPca>Threshold_KPCA)
KPCAComp=indexKPCA[0][0]

# Selection of the eigenvectors 
Yr_train_KPCA=YKpca[:,:KPCAComp]
Anr_train_KPCA=AnKpca[:,:KPCAComp]

# Construction matrix K for test
N = Xctrain.shape[0]
M = Xctest.shape[0]
InnerX = np.dot(Xctest,Xctrain.T)
tempTrain=np.sum(Xctrain**2,axis=1).reshape((1,N))
tempTest=np.sum(Xctest**2,axis=1).reshape((M,1))
NormTrain2 = np.repeat(tempTrain,M,axis=0)
NormTest2 = np.repeat(tempTest,N,axis=1) 
Norm = NormTest2+NormTrain2-2*InnerX
Norm[Norm<1e-10]=0    
Ktest=np.exp(-Norm/(2*gamma**2))

# Centering kernel test matrix
oneN=np.ones((N,N))/N
oneM=np.ones((M,M))/M
KcTest=Ktest-np.dot(oneM,Ktest)-np.dot(Ktest,oneN) + np.dot(np.dot(oneM,Ktest),oneN) # center kernel matrix
       
# Computation of the test scores using the eigenvectors computed with the training data-set        
Yr_test_KPCA=np.dot(KcTest,Anr_train_KPCA) 

print('KPCA uses ', Yr_train_KPCA.shape[0], ' features')


# Score
NN.fit(Yr_train_KPCA,Id_Train.ravel())
print('Percentage of correct answer using KPCA is ', NN.score(Yr_test_KPCA,Id_Test.ravel()))

In [ ]:
# For IMP
## KPCA (your implementation)
XXXXXXXXXXXXX

# ICA

In the next section you will evaluate ICA. Every image $x_i$ can be seen as a linear combination of basis images. ICA can be used in two different ways for face recognition. We can look for a set of statistically independent basis images $s_j$ (first architecture) or for a set of statistically independent coefficients $a_{j}$ (second architecture).

In the first architecture, we compute $X'=A'S'$, where every row of $X'$ is an image and the columns are pixels. Images are considered as random variables and we look for a set of statistically independent basis images contained in the rows of $S'$.

In the second architecture, we transpose the previous setting computing $X''=A''S''$, where every column of $X''$ is an image and rows are pixels. In this case, we consider the pixels as random variables and we look for a set of statistically independent coefficients contained in the rows of $S$ and a set of basis images in the columns of $A$.

Instead than using the original training data $X$ as input matrix, we are going to use the eigenvectors (first architecture) or the scores (second architecture) computed with PCA, namely $Y=XL$ (same notation as in the slides of the lecture). In this way, we reduce the computational time since the number of eigenvectors that account for 99\% of the variance of the training images (columns of $L$) is definitely lower than the number of pixels (columns of $X$). If you want, you can of course use the original data but it will take much more time to converge.

For the first architecture we will use $L^T$ as input matrix. In fact, we can notice that the PCA approximation of the matrix $X_{train}$, containing an image in every row, can be written as $\tilde{X} = YL^T$. If we use $L^T$ as input in the ICA algorithm we obtain $L^T=AS$, thus it follows that $\tilde{X}=YW^TS$ (since $A=W^{-1}=W^T$). The basis images are contained in the rows of $S$ and the coefficients used for evaluating the performance are instead contained in the rows of $Y_{train}W^T$ for the training set and in $Y_{test}W^T$ for the test set.

For the second architecture, we will instead use $Y^T$ as input matrix thus obtaining $Y^T=AS$. Remember that in the second architecture we want to apply the ICA algorithm to the transpose of $X_{train}$, namely $X^T=AS$. We can notice that, given the PCA transformation $Y=XL$, one can write $X \approx YL^T$ which entails $X^T \approx LY^T=LAS=LW^TS$. The columns of $LW^T$ contain the basis images whereas the columns of $S$ contain the statistically independent coefficients used to test the performance of the algorithm. The coefficients for the test set are in the columns of $S_{test}=W_{train}Y_{test}^T$.

NB: Here we used $X=X_c$ which means centered face images

**Questions:**
 

1.   (IMP+IMH) Look at the results of the two architecures. Which one is better ?
2.   (IMP+IMH) Looking at the basis images, in which case do they seem more 'real' ?
3.   (IMP - Optional) Implement the two architecture using your own imlementation of ICA (`FastICALecture`) instead than the one of scikit-learn (Be careful at the input data...scikit-learn wants a [observations, features] matrix) 



In [ ]:
#ICA - first architecture (scikit-learn implementation)
pca = PCA(random_state=1) # by fixing the random_state we are sure that results are always the same
YpcaTrain=pca.fit_transform(Xtrain)
UpcaTrain=pca.components_.T # we want PC on columns
var_explained_pca=pca.explained_variance_ratio_

# We use the PCA projection to speed up results
# Threshold defined as 99% of the variability
Threshold_PCA = 0.99
CumulativePca=np.cumsum(var_explained_pca)
indexPCA=np.argwhere(CumulativePca>Threshold_PCA)
PCAComp=indexPCA[0][0]

# Selection of the eigenvectors 
Yr_train_PCA=YpcaTrain[:,:PCAComp]
Ur_train_PCA=UpcaTrain[:,:PCAComp]
Yr_test_PCA=np.dot(Xctest,Ur_train_PCA)

ICA= FastICA(whiten=True, fun='exp', max_iter=30000, algorithm='parallel', tol=1e-4)
Yica=ICA.fit_transform(Ur_train_PCA)
Sica=Yica.T
Aica=ICA.mixing_
Wica=ICA.components_

Y_test_ICA= np.dot(Yr_test_PCA, Aica)
Y_train_ICA = np.dot(Yr_train_PCA,Aica)

print('ICA uses ', Y_train_ICA.shape[0], ' features')

# Plot the Eigenfaces
plotFaces(Sica,r,c,ncol=2,indeces=np.arange(0,10,1),title='ICA - first architecture')  

# Score
NN.fit(Y_train_ICA,Id_Train.ravel())
print('Percentage of correct answer using ICA arch.1 is ', NN.score(Y_test_ICA,Id_Test.ravel()))

In [ ]:
# For IMP
# ICA - First architecture (your implementation)
XXXXXXXXXXX

In [ ]:
## ICA
# Second architecture (scikit-learn implementation)
pca = PCA(random_state=1) # by fixing the random_state we are sure that results are always the same
YpcaTrain=pca.fit_transform(Xtrain)
UpcaTrain=pca.components_.T # we want PC on columns
var_explained_pca=pca.explained_variance_ratio_

# Threshold defined as 99% of the variability 
Threshold_PCA = 0.99
CumulativePca=np.cumsum(var_explained_pca)
indexPCA=np.argwhere(CumulativePca>Threshold_PCA)
PCAComp=indexPCA[0][0]

# Selection of the eigenvectors 
Yr_train_PCA=YpcaTrain[:,:PCAComp]
Ur_train_PCA=UpcaTrain[:,:PCAComp]
Yr_test_PCA=np.dot(Xctest,Ur_train_PCA)

ICA= FastICA(whiten=True, fun='exp', max_iter=30000, tol=1e-4, algorithm='parallel', random_state=1)
Yica=ICA.fit_transform(Yr_train_PCA)
S_train_ICA=Yica.T
W_train_ICA=ICA.components_

ICAFAces=np.dot(Ur_train_PCA,W_train_ICA.T) 
Y_train_ICA=S_train_ICA
Y_test_ICA=np.dot(W_train_ICA,Yr_test_PCA.T)

# Plot the ICA-faces
plotFaces(ICAFAces.T,r,c,ncol=2,indeces=np.arange(0,10,1),title='ICA-faces')      

print('ICA uses ', Y_train_ICA.shape[0], ' features')

# Score ICA

NN.fit(Y_train_ICA.T,Id_Train.ravel())
print('Percentage of correct answer using ICA is ', NN.score(Y_test_ICA.T,Id_Test.ravel()))

In [ ]:
# For IMP
## ICA - second architecture (your implementation)
XXXXXXXXXXX

# NNMF

Here you will test Non-negative Matrix factorization. The basis images of the training are in the matrix $W_{train}$ and the scores (or coefficients) to test the performance in $H_{train}$. The test scores are computed as $H_{test}=W_{train}^{-1}X_{test}$.

**Question**


1.   (IMP+IMH) Plot the basis images and compare them with respect to the basis images obtained using PCA and ICA. What can you say ?
2.   (IMP+IMH) What about the performances of NNMF, i.e. computational time and classification accuracy ? Is it better or worse than the other methods ? Why ?
3.   (IMP) Implement your own implementation in `NNMFLecture` following the lecture slides. Complete the missing lines (`XXXXXXXXXX`) 

In [ ]:
# NNMF (scikit-learn implementation)
Ncomponents=100 # choose a number of componenents (r in the slides)
model = NMF(init='random', solver='mu', n_components=Ncomponents, tol=1e-3,max_iter=300, random_state=0)
WtrainNNMF = model.fit_transform(Xtrain.T)
HtrainNNMF = model.components_

plotFaces(WtrainNNMF.T,r,c,ncol=2,indeces=np.arange(0,10,1),title='NNMF-faces') 

Htest_nnmf = np.dot(LA.pinv(WtrainNNMF),Xtest.T)

print('NNMF uses ', Ncomponents, ' features')

# Score
NN.fit(HtrainNNMF.T,Id_Train.ravel())
print('Percentage of correct answer using NNMF is ', NN.score(Htest_nnmf.T,Id_Test))


In [ ]:
# For IMP
def NNMFLecture(X,r=None,N_Iter=1000,tolerance=1e-3,plot_evolution=1):
    '''
    Inputs: 
    %           X: is a [dxN] matrix. Every column (x) is an observation and every
    %           row consists of features.
    %
    %           r: size of the matrices W and H
    %
    %           (Optional) N_Iter: maximum number of iterations
    %
    %           (Optional) tolerance: convergence criteria threshold
    %
    %           (Optional) plot_evolution: plot evolution convergence criteria
    %
    % Outputs:
    %           W: is a [d x r] matrix containing the basis images in its
    %           columns
    %           
    %           H: is a [r x N] matrix containing the loadings (h) in its columns
    %           of the linear combination: x=Wh 
    %
  '''
    if r is None:
        r=X.shape[0]
        
    # Test for positive values
    if np.min(X) < 0:
        raise NameError('Input matrix X has negative values !')      

    # Size
    d,N=X.shape
   
    # Initialization
    W=np.random.rand(d,r)
    H=np.random.rand(r,N)     
    
    # parameters for convergence
    k = 0
    delta = np.inf
    eps=np.finfo(float).eps
    evolutionDelta=[]
 
    while delta > tolerance and k < N_Iter:
        
        # multiplicative method      
        for i in range(20):
            W = np.divide(XXXXXXXX + eps)           
            
        H = np.divide(XXXXXXXXXX + eps)

        # Convergence indices
        k = k + 1           
        diff=X-np.dot(W,H)     
        delta = LA.norm(diff,'fro') / LA.norm(X,'fro') # sqrt(trace(diff'*diff)) / sqrt(trace(X'*X))
        evolutionDelta.append(delta)
        
        if k==1 or k%100==0:
            print('Iteration NNMF number ', k, ' out of ', N_Iter , ', delta = ', delta, ', error (norm delta): ', LA.norm(diff))
     
    if k==N_Iter:
        print('Maximum number of iterations reached ! delta = ', delta)
    else:
        print('Convergence achieved ( delta = ', delta, ') in ', k, ' iterations')
    
    if plot_evolution==1:
        plt.figure(figsize=(6, 6))
        plt.plot(range(k),evolutionDelta,'bx--', linewidth=4, markersize=12)  
        plt.title('Evolution of error - NNMF')
        plt.show()
    
    return W,H

In [ ]:
# For IMP
# NNMF (test your own implementation)
XXXXXXXXXXX

Here, we check that using the original data for ICA is definitely too long. In scikit-learn we directly select the number of components $p$. However, results are less satisfactory than using PCA as before or too long.

In [ ]:
# Shuffle data randomly and use only 300 components (p=300)
indeces=np.arange(data.shape[0]) # Integers from 0 to N-1
np.random.shuffle(indeces)
sample=data[indeces,:]

#first architecture (scikit-learn implementation)
ICA= FastICA(whiten=True, fun='exp', max_iter=30000, algorithm='parallel', tol=1e-4, random_state=1, n_components=300)
Yica=ICA.fit_transform(sample)
Sica=Yica.T
Aica=ICA.mixing_
Wica=ICA.components_

# Plot the Eigenfaces
plotFaces(Sica,r,c,ncol=2,indeces=np.arange(0,10,1),title='ICA basis images (first architecture)')  


In [ ]:
#second architecture (scikit-learn implementation)
ICA= FastICA(whiten=True, fun='exp', max_iter=30000, algorithm='parallel', tol=1e-4, random_state=1, n_components=300)
Yica=ICA.fit_transform(data.T)
Sica=Yica.T
Aica=ICA.mixing_
Wica=ICA.components_

# Plot the Eigenfaces
plotFaces(Aica.T,r,c,ncol=2,indeces=np.arange(0,10,1),title='ICA basis images (second architecture)')  